In [ ]:
!pip install datasets torchvision Pillow

In [ ]:
from datasets import load_dataset

# Load the modern logo dataset
dataset = load_dataset("logo-wizard/modern-logo-dataset")

# Display the structure of the dataset
print(dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 803
    })
})


In [ ]:
# Display the first few samples
for i in range(5):
    print(dataset['train'][i])


{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=904x938 at 0x7A96B895BD60>, 'text': '"a logo of coffee shop, take-away coffee cardboard glass with white and brown stripes and dark brown lid, coffee circle with three cream drops, white background, brown foreground, minimalism, modern"'}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=968x928 at 0x7A96B89CE560>, 'text': '"a logo of coffee shop, White round background with black rim, cup, pretzel, horizontal stripe and cookery lettering, tan background, snow, darkslategray foreground, minimalism, modern"'}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1060x1040 at 0x7A96B89CE650>, 'text': '"a logo of coffee shop, image of a filled cup with steam in a square, white background, black, darkgray foreground, minimalism, modern"'}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=588x580 at 0x7A96B895B7F0>, 'text': '"a logo of cafe restaurant bar pizzeria with a slice of pizza at t

In [ ]:
# Check the features of the dataset
print(dataset['train'].features)

# Display a sample item to see its structure
print(dataset['train'][0])


{'image': Image(mode=None, decode=True, id=None), 'text': Value(dtype='string', id=None)}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=904x938 at 0x7A96B89CE740>, 'text': '"a logo of coffee shop, take-away coffee cardboard glass with white and brown stripes and dark brown lid, coffee circle with three cream drops, white background, brown foreground, minimalism, modern"'}


In [ ]:
import torchvision.transforms as transforms

# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize the image to 128x128
    transforms.ToTensor(),            # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])


In [ ]:
from datasets import load_dataset
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage

# Load the modern logo dataset
dataset = load_dataset("logo-wizard/modern-logo-dataset")

# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize the image to 128x128
    transforms.ToTensor(),            # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

def preprocess_data(dataset):
    images, descriptions = [], []

    for item in dataset['train']:
        image = item['image']  # This is already a PIL image
        if image.mode != 'RGB':
            image = image.convert('RGB')  # Convert to RGB if not already
        image = transform(image)  # Apply transformations
        images.append(image)
        descriptions.append(item['text'])  # Use the correct key for descriptions

    return images, descriptions

# Call the preprocessing function
images, descriptions = preprocess_data(dataset)

# Instantiate the conversion transformation
to_pil = ToPILImage()

# Display a sample image and description
sample_index = 0
image_to_show = to_pil(images[sample_index])
image_to_show.show()
print(descriptions[sample_index])


"a logo of coffee shop, take-away coffee cardboard glass with white and brown stripes and dark brown lid, coffee circle with three cream drops, white background, brown foreground, minimalism, modern"


In [ ]:
# Check the number of samples
print(f'Number of images: {len(images)}')
print(f'Number of descriptions: {len(descriptions)}')

# Display a sample image and description
sample_index = 0
image_to_show = to_pil(images[sample_index])  # Convert tensor to PIL image
image_to_show.show()
print(descriptions[sample_index])



Number of images: 803
Number of descriptions: 803
"a logo of coffee shop, take-away coffee cardboard glass with white and brown stripes and dark brown lid, coffee circle with three cream drops, white background, brown foreground, minimalism, modern"


In [ ]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, text_embedding_size=100):  # Adjust this based on your actual embedding size
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100 + text_embedding_size, 256),  # Update this size
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 49152),  # Update output size if needed
            nn.Tanh()
        )

    def forward(self, noise, text_embedding):
        combined_input = torch.cat((noise, text_embedding), dim=1)
        return self.model(combined_input)




In [ ]:
import torch
import torch.nn as nn

In [ ]:
def train_model(generator, images, descriptions, epochs=10, batch_size=32):
    criterion = nn.BCELoss()  # Define loss function
    optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002)

    for epoch in range(epochs):
        for i in range(0, len(images), batch_size):
            # Get batch of images
            batch_images = images[i:i + batch_size]
            batch_images = torch.stack(batch_images).to(device)  # Move to GPU if available

            # Create random noise
            noise = torch.randn(batch_size, 100).to(device)  # Adjust noise size
            generated_images = generator(noise)

            # Calculate loss (you'll need to define how to compare generated with real)
            # Loss calculation logic goes here

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{epochs} completed')

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator.to(device)

# Start training (commented out for now; will implement loss calculation first)
# train_model(generator, images, descriptions)


Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=49152, bias=True)
    (5): Tanh()
  )
)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(49152, 512),  # Adjust based on your image size
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),  # Output a single value for real/fake
            nn.Sigmoid()  # Use Sigmoid for binary classification
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
discriminator = Discriminator().to(device)


In [ ]:
def train_model(generator, discriminator, images, descriptions, epochs=10, batch_size=32):
    for epoch in range(epochs):
        d_loss_epoch = 0
        g_loss_epoch = 0
        total_batches = len(images) // batch_size

        for i in range(0, len(images), batch_size):
            # Get a batch of images
            batch_images = images[i:i + batch_size]
            if len(batch_images) < batch_size:
                continue  # Skip if the last batch is smaller

            batch_images = torch.stack(batch_images).to(device)

            # Create random noise for the generator
            noise = torch.randn(batch_images.size(0), 100).to(device)  # Use current batch size
            generated_images = generator(noise)

            # Create labels for real and fake images
            real_labels = torch.ones(batch_images.size(0), 1).to(device)
            fake_labels = torch.zeros(batch_images.size(0), 1).to(device)

            # Train Discriminator
            optimizer_d.zero_grad()
            outputs = discriminator(batch_images.view(batch_images.size(0), -1))
            d_loss_real = criterion(outputs, real_labels)

            outputs = discriminator(generated_images.detach().view(generated_images.size(0), -1))
            d_loss_fake = criterion(outputs, fake_labels)

            d_loss = d_loss_real + d_loss_fake
            d_loss.backward()
            optimizer_d.step()

            # Train Generator
            optimizer_g.zero_grad()
            outputs = discriminator(generated_images.view(generated_images.size(0), -1))
            g_loss = criterion(outputs, real_labels)

            g_loss.backward()
            optimizer_g.step()

            # Accumulate losses for the epoch
            d_loss_epoch += d_loss.item()
            g_loss_epoch += g_loss.item()

            # Log losses and progress
            if i % (batch_size * 10) == 0:  # Print every 10 batches
                print(f'Epoch [{epoch + 1}/{epochs}], Step [{i // batch_size + 1}/{total_batches}], '
                      f'D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}')

        # Print average losses for the epoch
        print(f'End of Epoch [{epoch + 1}/{epochs}], Avg D Loss: {d_loss_epoch / total_batches:.4f}, Avg G Loss: {g_loss_epoch / total_batches:.4f}')

# Call the function to start training
train_model(generator, discriminator, images, descriptions, epochs=10, batch_size=32)


Epoch [1/10], Step [1/25], D Loss: 1.3553, G Loss: 0.7597
Epoch [1/10], Step [11/25], D Loss: 0.3521, G Loss: 1.4991
Epoch [1/10], Step [21/25], D Loss: 0.4122, G Loss: 1.3883
End of Epoch [1/10], Avg D Loss: 0.5082, Avg G Loss: 1.3155
Epoch [2/10], Step [1/25], D Loss: 0.1627, G Loss: 2.3886
Epoch [2/10], Step [11/25], D Loss: 0.2948, G Loss: 1.7007
Epoch [2/10], Step [21/25], D Loss: 0.5402, G Loss: 0.9791
End of Epoch [2/10], Avg D Loss: 0.3507, Avg G Loss: 1.6763
Epoch [3/10], Step [1/25], D Loss: 0.2997, G Loss: 1.6016
Epoch [3/10], Step [11/25], D Loss: 0.1240, G Loss: 2.7200
Epoch [3/10], Step [21/25], D Loss: 0.3504, G Loss: 2.6354
End of Epoch [3/10], Avg D Loss: 0.2688, Avg G Loss: 2.5884
Epoch [4/10], Step [1/25], D Loss: 0.0152, G Loss: 5.6730
Epoch [4/10], Step [11/25], D Loss: 0.1697, G Loss: 3.6843
Epoch [4/10], Step [21/25], D Loss: 0.1441, G Loss: 2.4302
End of Epoch [4/10], Avg D Loss: 0.1603, Avg G Loss: 4.0194
Epoch [5/10], Step [1/25], D Loss: 0.0720, G Loss: 3.510

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample text encoder using TF-IDF
vectorizer = TfidfVectorizer(max_features=100)  # Adjust max_features as needed
text_samples = ["Coffee Shop Logo", "Modern Logo", "Tech Company Logo", "Fashion Brand Logo"]  # Add more samples as needed
vectorizer.fit(text_samples)

def encode_texts(texts):
    return vectorizer.transform(texts).toarray()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Define your vectorizer globally
vectorizer = TfidfVectorizer(max_features=100)  # Ensure max_features is set to 100

# Fit the vectorizer on training text samples
text_samples = ["Coffee Shop Logo", "Modern Logo", "Tech Logo", "Artistic Logo"]  # Example training samples
vectorizer.fit(text_samples)

def encode_text(text):
    text_vector = vectorizer.transform([text]).toarray()  # Transform text to array
    return torch.FloatTensor(text_vector).to(device)  # Convert to tensor and move to device


In [ ]:
def generate_logo(generator, text):
    text_vector = encode_text(text)  # Encode the text
    noise = torch.randn(1, 100).to(device)  # Generate noise of the correct size
    input_vector = text_vector + noise  # Combine text vector with noise
    generated_image = generator(input_vector)  # Generate image
    return generated_image.view(-1, 3, 128, 128).cpu().detach().numpy()  # Reshape and move to CPU


In [ ]:
# Generate a logo for "Coffee Shop Logo"
logo_image = generate_logo(generator, "Coffee Shop Logo")

# Display the generated logo
display_image(logo_image)



text_vector shape: torch.Size([1, 6]), noise shape: torch.Size([1, 100])


RuntimeError: The size of tensor a (6) must match the size of tensor b (100) at non-singleton dimension 1